In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
movies = pd.read_csv('./ETL/movies_limpio.csv')
movies.shape

C:\Users\clari\AppData\Local\Temp\ipykernel_3996\3492386156.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('./ETL/movies_limpio.csv')


(45291, 23)

In [9]:
recomendacion = movies[['title', 'overview', 'popularity']]  # elijo las columnas relevantes para el modelo
recomendacion = recomendacion.dropna()  # elimino las filas que contienen valores nulos

# Convierto la columna "popularity" a tipo float
recomendacion['popularity'] = recomendacion['popularity'].astype(float)

# Ordeno por popularidad en orden descendente 
# (ascending=False=547.488298 popularity alto)  y  (ascending=True=213.849907 popularity bajo)
recomendacion = recomendacion.sort_values(by='popularity', ascending=False)
recomendacion = recomendacion[recomendacion['popularity'] != 0]#saco los que son valor 0

#filtro para achicar la muestra ya que ocupa mucho espacio al crear la matriz. 
# Dejando asi las peliculas con popularity > 10 (asi el usuario recibe una recomendacion de calidad)
recomendacion_filtrado = recomendacion[recomendacion['popularity'] > 10]
#print(recomendacion_filtrado)

#reseteo el index para que al consultar una pelicula el index coincida con la totalidad de las peliculas filtradas
recomendacion_filtrado=recomendacion_filtrado.reset_index()

# Guardar en un archivo CSV
#recomendacion_filtrado.to_csv('peliculas_filtradas.csv', index=False)




In [10]:
recomendacion_filtrado.shape

(3016, 4)

In [15]:
recomendacion_filtrado.to_csv('peliculas_filtradas.csv', index=False)

In [16]:
recomendacion_filtrado

,title,overview,popularity
0,Minions,"Minions Stuart, Kevin and Bob are recruited by...",547.488298
1,Wonder Woman,An Amazon princess comes to the world of Man t...,294.337037
2,Beauty and the Beast,A live-action adaptation of Disney's version o...,287.253654
3,Baby Driver,After being coerced into working for a crime b...,228.032744
4,Big Hero 6,The special bond that develops between plus-si...,213.849907
...,...,...,...
3011,The Dead Lands,"Hongi, a Maori chieftain’s teenage son, must a...",10.001517
3012,All About My Mother,A single mother in Madrid sees her only son di...,10.000915
3013,Project X,A young inductee into the military is given th...,10.000775
3014,Scream 4,"Sidney Prescott, now the author of a self-help...",10.000721


IMPORTANTE: Debido a la poca capacidad de Render, tuve que reducir el dataset a 3mil datos.

In [17]:
recomendacion_filtrado['title'][20]

'The Bad Batch'

In [18]:

def recomendar_peliculas(titulo_pelicula):
    df = pd.DataFrame(recomendacion_filtrado)
    num_similares = 5
    
    # Filtro el df por el título de la peli solicitada
    pelicula_filtro = df[df["title"] == titulo_pelicula]
    
    if pelicula_filtro.empty:
        print("No se encontró ninguna película con ese título.")
        return []
    
    # Obtengo el índice de la película filtrada
    target_movie_index = pelicula_filtro.index[0]

    # Creo las representaciones numéricas de las descripciones de películas
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df["overview"])

    # Calculo la similitud del coseno entre las descripciones de películas
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Obtengo los índices de las películas más similares
    similar_movies_indices = similarity_matrix[target_movie_index].argsort()[::-1][1:num_similares+1]

    # Obtengo las películas similares del df
    peliculas_similares = df.iloc[similar_movies_indices]["title"].tolist()
    
    return f" Las peliculas similares para el titulo '{titulo_pelicula.capitalize()}' son:  {peliculas_similares}."


In [19]:
recomendar_peliculas('The Bad Batch')

" Las peliculas similares para el titulo 'The bad batch' son:  ['Turbo Kid', 'Beyond the Reach', 'Bone Tomahawk', 'Stranger Than Fiction', 'Mine']."

In [11]:
#pruebo si me funciona con la primer pelicula de la lista
data = recomendacion_filtrado
df = pd.DataFrame(data)

# Creoar representaciones numéricas de las descripciones de películas
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["overview"])

# Calculo la similitud del coseno entre las descripciones de películas
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Obtengo las películas más similares a la pelicula (primer película en la lista)
target_movie_index = 0
similar_movies_indices = similarity_matrix[target_movie_index].argsort()[::-1][1:6]


print("Películas similares a", df.iloc[target_movie_index]["title"])
for index in similar_movies_indices:
    print("- ", df.iloc[index]["title"])


Películas similares a Minions
-  Despicable Me 2
-  Banana
-  Stuart Little 3: Call of the Wild
-  Stuart Little 2
-  Mary of Scotland
